In [ ]:
from torchvision import models, transforms
from torchsummary import summary
import torch
import numpy as np
import cv2
import PIL
import matplotlib.pyplot as plt

from utils import *
from src import *

In [ ]:
model = models.resnet50(pretrained=True)
class_names = np.loadtxt('imagenet_classes.txt', str, delimiter='\t')

In [ ]:
img = get_image_torch('images/husky.jpg', 224, 224)
cam_husky_our = get_gradcam(img, model, model.layer4[-1].conv3)
overlay_plot_torch(img, cam_husky_our, 0.5, 'images_prezentacja/gradcam_husky.png')

In [ ]:
img = get_image_torch('images/precelek.jpg', 224, 224)
cam_pretzel_our = get_gradcam(img, model, model.layer4[-1].conv3)
overlay_plot_torch(img, cam_pretzel_our, 0.5, 'images_prezentacja/gradcam_precel.png')

In [ ]:
from pytorch_grad_cam import GradCAM, HiResCAM, AblationCAM, ScoreCAM, GradCAMPlusPlus, XGradCAM
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image
from torchvision.models import resnet50
from utils import *
from src import *

In [ ]:
model = resnet50(pretrained=True)
target_layers = [model.layer4[-1].conv3]

In [ ]:
input_tensor = get_image_torch('images/husky.jpg', 224, 224).unsqueeze(0)
targets = [ClassifierOutputTarget(250)]

with GradCAM(model=model, target_layers=target_layers) as cam:
  cam_husky_lib = cam(input_tensor=input_tensor, targets=targets).squeeze(0)
overlay_plot_torch(input_tensor.squeeze(), cam_husky_lib, 0.5, 'images_prezentacja/gradcam_husky_lib.png')

input_tensor = get_image_torch('images/precelek.jpg', 224, 224).unsqueeze(0)
targets = [ClassifierOutputTarget(932)]

with GradCAM(model=model, target_layers=target_layers) as cam:
  cam_pretzel_lib = cam(input_tensor=input_tensor, targets=targets).squeeze(0)
overlay_plot_torch(input_tensor.squeeze(), cam_pretzel_lib, 0.5, 'images_prezentacja/gradcam_precel_lib.png')

In [ ]:
cam_husky_our = cv2.resize(cam_husky_our, (224, 224))
cam_pretzel_our = cv2.resize(cam_pretzel_our, (224, 224))

In [ ]:
np.mean((cam_husky_our - cam_husky_lib)**2), np.mean((cam_pretzel_our - cam_pretzel_lib)**2)

In [ ]:
from skimage.metrics import structural_similarity as ssim
# 1 identyczne 
# -1 rozna

ssim(cam_husky_our, cam_husky_lib, data_range=cam_husky_our.max() - cam_husky_our.min()), ssim(cam_pretzel_our, cam_pretzel_lib, data_range=cam_pretzel_our.max() - cam_pretzel_our.min())

In [ ]:
np.mean(np.abs(cam_husky_our - cam_husky_lib)), np.mean(np.abs(cam_pretzel_our - cam_pretzel_lib))

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
# 1 identyczne
# 0 brak

cosine_similarity(cam_husky_our.reshape(1, -1), cam_husky_lib.reshape(1, -1)), cosine_similarity(cam_pretzel_our.reshape(1, -1), cam_pretzel_lib.reshape(1, -1))